## Setup

In [ ]:
import numpy as np
from numba import cuda
import math
import time

print(f"CUDA available: {cuda.is_available()}")
if cuda.is_available():
    device = cuda.get_current_device()
    print(f"Device: {device.name}")

---

## Part 1: The Kernel Fusion Problem

### Why Fusion Matters

Consider computing: `z = a*x + b*y + c`

```
Approach 1: Separate Kernels
━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Kernel 1: temp1 = a * x        # Read x, Write temp1
Kernel 2: temp2 = b * y        # Read y, Write temp2  
Kernel 3: temp3 = temp1 + temp2 # Read temp1, temp2, Write temp3
Kernel 4: z = temp3 + c        # Read temp3, Write z

Memory traffic: MANY reads and writes!
Kernel launches: 4 (overhead for each)

Approach 2: Fused Kernel
━━━━━━━━━━━━━━━━━━━━━━━━
Kernel: z = a*x + b*y + c      # Read x, y, Write z

Memory traffic: Minimal (2 reads, 1 write)
Kernel launches: 1
```

In [ ]:
# Separate kernels (non-fused)
@cuda.jit
def scalar_multiply(a, x, out, n):
    """out = a * x"""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(tid, n, stride):
        out[i] = a * x[i]

@cuda.jit
def vector_add(a, b, out, n):
    """out = a + b"""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(tid, n, stride):
        out[i] = a[i] + b[i]

@cuda.jit
def scalar_add(x, c, out, n):
    """out = x + c"""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(tid, n, stride):
        out[i] = x[i] + c

In [ ]:
# Fused kernel
@cuda.jit
def fused_axpbyc(a, x, b, y, c, z, n):
    """z = a*x + b*y + c (all in one kernel)"""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(tid, n, stride):
        z[i] = a * x[i] + b * y[i] + c

In [ ]:
# Compare performance
n = 10_000_000
a, b, c = 2.0, 3.0, 1.0

x = np.random.rand(n).astype(np.float32)
y = np.random.rand(n).astype(np.float32)

d_x = cuda.to_device(x)
d_y = cuda.to_device(y)

blocks, threads = 256, 256
iterations = 100

# Non-fused approach
d_temp1 = cuda.device_array(n, dtype=np.float32)
d_temp2 = cuda.device_array(n, dtype=np.float32)
d_temp3 = cuda.device_array(n, dtype=np.float32)
d_z_nonfused = cuda.device_array(n, dtype=np.float32)

# Warmup
scalar_multiply[blocks, threads](a, d_x, d_temp1, n)
scalar_multiply[blocks, threads](b, d_y, d_temp2, n)
vector_add[blocks, threads](d_temp1, d_temp2, d_temp3, n)
scalar_add[blocks, threads](d_temp3, c, d_z_nonfused, n)
cuda.synchronize()

start = time.perf_counter()
for _ in range(iterations):
    scalar_multiply[blocks, threads](a, d_x, d_temp1, n)
    scalar_multiply[blocks, threads](b, d_y, d_temp2, n)
    vector_add[blocks, threads](d_temp1, d_temp2, d_temp3, n)
    scalar_add[blocks, threads](d_temp3, c, d_z_nonfused, n)
cuda.synchronize()
nonfused_time = (time.perf_counter() - start) / iterations * 1000

# Fused approach
d_z_fused = cuda.device_array(n, dtype=np.float32)

fused_axpbyc[blocks, threads](a, d_x, b, d_y, c, d_z_fused, n)
cuda.synchronize()

start = time.perf_counter()
for _ in range(iterations):
    fused_axpbyc[blocks, threads](a, d_x, b, d_y, c, d_z_fused, n)
cuda.synchronize()
fused_time = (time.perf_counter() - start) / iterations * 1000

# Verify correctness
expected = a * x + b * y + c
result_nonfused = d_z_nonfused.copy_to_host()
result_fused = d_z_fused.copy_to_host()

print(f"z = {a}*x + {b}*y + {c}")
print(f"N = {n:,}\n")
print(f"Non-fused (4 kernels): {nonfused_time:.3f} ms")
print(f"Fused (1 kernel):      {fused_time:.3f} ms")
print(f"Speedup:               {nonfused_time/fused_time:.2f}x")
print(f"\nCorrectness: {'✓' if np.allclose(result_fused, expected) else '✗'}")

---

## Part 2: Memory Traffic Analysis

### Counting Memory Operations

In [ ]:
def analyze_memory_traffic(n, sizeof=4):
    """Analyze memory traffic for z = a*x + b*y + c"""
    
    # Non-fused approach
    # Kernel 1: temp1 = a * x
    k1_read = n * sizeof   # read x
    k1_write = n * sizeof  # write temp1
    
    # Kernel 2: temp2 = b * y
    k2_read = n * sizeof   # read y
    k2_write = n * sizeof  # write temp2
    
    # Kernel 3: temp3 = temp1 + temp2
    k3_read = 2 * n * sizeof   # read temp1, temp2
    k3_write = n * sizeof      # write temp3
    
    # Kernel 4: z = temp3 + c
    k4_read = n * sizeof   # read temp3
    k4_write = n * sizeof  # write z
    
    nonfused_total = (k1_read + k1_write + k2_read + k2_write + 
                      k3_read + k3_write + k4_read + k4_write)
    
    # Fused approach
    # Single kernel: z = a*x + b*y + c
    fused_read = 2 * n * sizeof   # read x, y
    fused_write = n * sizeof      # write z
    fused_total = fused_read + fused_write
    
    return nonfused_total, fused_total

n = 10_000_000
nonfused_bytes, fused_bytes = analyze_memory_traffic(n)

print(f"Memory Traffic Analysis (N={n:,})")
print(f"{'='*45}")
print(f"Non-fused: {nonfused_bytes/1e9:.2f} GB")
print(f"Fused:     {fused_bytes/1e9:.2f} GB")
print(f"Reduction: {nonfused_bytes/fused_bytes:.1f}x less memory traffic")

---

## Part 3: Common Fusion Patterns

### Pattern 1: Fused Multiply-Add (FMA)

This is so common that GPUs have hardware support for it!

In [ ]:
@cuda.jit
def fused_multiply_add(a, b, c, out, n):
    """out = a * b + c (element-wise FMA)"""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(tid, n, stride):
        # GPU actually has a single FMA instruction!
        out[i] = a[i] * b[i] + c[i]

# Test
n = 1_000_000
a = np.random.rand(n).astype(np.float32)
b = np.random.rand(n).astype(np.float32)
c = np.random.rand(n).astype(np.float32)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.to_device(c)
d_out = cuda.device_array(n, dtype=np.float32)

fused_multiply_add[256, 256](d_a, d_b, d_c, d_out, n)
result = d_out.copy_to_host()
expected = a * b + c

print(f"FMA (a*b + c): {'✓' if np.allclose(result, expected) else '✗'}")

### Pattern 2: Activation Functions with Scaling

In [ ]:
@cuda.jit
def fused_scale_relu(x, scale, out, n):
    """out = ReLU(scale * x) = max(0, scale * x)"""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(tid, n, stride):
        val = scale * x[i]
        out[i] = max(0.0, val)

@cuda.jit
def fused_linear_sigmoid(x, weight, bias, out, n):
    """out = sigmoid(weight * x + bias)"""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(tid, n, stride):
        linear = weight * x[i] + bias
        out[i] = 1.0 / (1.0 + math.exp(-linear))

@cuda.jit
def fused_batchnorm_relu(x, mean, var, gamma, beta, epsilon, out, n):
    """BatchNorm + ReLU in one kernel."""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(tid, n, stride):
        # Normalize
        normalized = (x[i] - mean) / math.sqrt(var + epsilon)
        # Scale and shift
        bn_out = gamma * normalized + beta
        # ReLU
        out[i] = max(0.0, bn_out)

In [ ]:
# Benchmark: separate vs fused BatchNorm+ReLU

@cuda.jit
def batchnorm_only(x, mean, var, gamma, beta, epsilon, out, n):
    """BatchNorm without ReLU."""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(tid, n, stride):
        normalized = (x[i] - mean) / math.sqrt(var + epsilon)
        out[i] = gamma * normalized + beta

@cuda.jit
def relu_only(x, out, n):
    """ReLU only."""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(tid, n, stride):
        out[i] = max(0.0, x[i])

n = 10_000_000
x = np.random.randn(n).astype(np.float32)
mean, var = 0.0, 1.0
gamma, beta, epsilon = 1.0, 0.0, 1e-5

d_x = cuda.to_device(x)
d_temp = cuda.device_array(n, dtype=np.float32)
d_out_sep = cuda.device_array(n, dtype=np.float32)
d_out_fused = cuda.device_array(n, dtype=np.float32)

blocks, threads = 256, 256
iterations = 100

# Separate kernels
batchnorm_only[blocks, threads](d_x, mean, var, gamma, beta, epsilon, d_temp, n)
relu_only[blocks, threads](d_temp, d_out_sep, n)
cuda.synchronize()

start = time.perf_counter()
for _ in range(iterations):
    batchnorm_only[blocks, threads](d_x, mean, var, gamma, beta, epsilon, d_temp, n)
    relu_only[blocks, threads](d_temp, d_out_sep, n)
cuda.synchronize()
separate_time = (time.perf_counter() - start) / iterations * 1000

# Fused kernel
fused_batchnorm_relu[blocks, threads](d_x, mean, var, gamma, beta, epsilon, d_out_fused, n)
cuda.synchronize()

start = time.perf_counter()
for _ in range(iterations):
    fused_batchnorm_relu[blocks, threads](d_x, mean, var, gamma, beta, epsilon, d_out_fused, n)
cuda.synchronize()
fused_time = (time.perf_counter() - start) / iterations * 1000

print(f"BatchNorm + ReLU (N={n:,})")
print(f"Separate (2 kernels): {separate_time:.3f} ms")
print(f"Fused (1 kernel):     {fused_time:.3f} ms")
print(f"Speedup:              {separate_time/fused_time:.2f}x")

### Pattern 3: Polynomial Evaluation

In [ ]:
@cuda.jit
def fused_polynomial(x, a, b, c, d, out, n):
    """Evaluate ax³ + bx² + cx + d using Horner's method."""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(tid, n, stride):
        xi = x[i]
        # Horner's method: ((a*x + b)*x + c)*x + d
        out[i] = ((a * xi + b) * xi + c) * xi + d

# Compare with separate operations
@cuda.jit
def poly_step1(x, a, out, n):  # a*x³
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(tid, n, stride):
        out[i] = a * x[i] * x[i] * x[i]

@cuda.jit  
def poly_step2(x, b, out, n):  # b*x²
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(tid, n, stride):
        out[i] = b * x[i] * x[i]

# ... and so on for each term

In [ ]:
# Test polynomial evaluation
n = 1_000_000
x = np.random.rand(n).astype(np.float32)
a, b, c, d = 2.0, -3.0, 1.0, 5.0

d_x = cuda.to_device(x)
d_out = cuda.device_array(n, dtype=np.float32)

fused_polynomial[256, 256](d_x, a, b, c, d, d_out, n)
result = d_out.copy_to_host()
expected = a * x**3 + b * x**2 + c * x + d

print(f"Polynomial: {a}x³ + {b}x² + {c}x + {d}")
print(f"Correct: {'✓' if np.allclose(result, expected) else '✗'}")

---

## Part 4: Advanced Fusion - Multi-Input Operations

In [ ]:
@cuda.jit
def fused_weighted_average_3(x1, w1, x2, w2, x3, w3, out, n):
    """Weighted average of 3 vectors: (w1*x1 + w2*x2 + w3*x3) / (w1+w2+w3)"""
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    weight_sum = w1 + w2 + w3
    
    for i in range(tid, n, stride):
        out[i] = (w1 * x1[i] + w2 * x2[i] + w3 * x3[i]) / weight_sum

@cuda.jit
def fused_softmax_cross_entropy(logits, targets, loss, n):
    """
    Simplified softmax cross-entropy for binary classification.
    loss = -sum(target * log(sigmoid(logit)) + (1-target) * log(1-sigmoid(logit)))
    """
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    
    for i in range(tid, n, stride):
        logit = logits[i]
        target = targets[i]
        
        # Numerically stable sigmoid
        if logit >= 0:
            prob = 1.0 / (1.0 + math.exp(-logit))
        else:
            exp_logit = math.exp(logit)
            prob = exp_logit / (1.0 + exp_logit)
        
        # Cross-entropy (avoid log(0))
        eps = 1e-7
        prob = max(eps, min(1 - eps, prob))
        loss[i] = -(target * math.log(prob) + (1 - target) * math.log(1 - prob))

In [ ]:
# Test weighted average
n = 100_000
x1 = np.random.rand(n).astype(np.float32)
x2 = np.random.rand(n).astype(np.float32)
x3 = np.random.rand(n).astype(np.float32)
w1, w2, w3 = 0.5, 0.3, 0.2

d_x1 = cuda.to_device(x1)
d_x2 = cuda.to_device(x2)
d_x3 = cuda.to_device(x3)
d_out = cuda.device_array(n, dtype=np.float32)

fused_weighted_average_3[256, 256](d_x1, w1, d_x2, w2, d_x3, w3, d_out, n)
result = d_out.copy_to_host()
expected = (w1 * x1 + w2 * x2 + w3 * x3) / (w1 + w2 + w3)

print(f"Weighted Average: {'✓' if np.allclose(result, expected) else '✗'}")

---

## Part 5: When NOT to Fuse

### Fusion Trade-offs

```
✓ DO Fuse When:
• Operations are element-wise (same index)
• Intermediate results are only used once
• Memory bandwidth is the bottleneck
• Operations are simple (not register-heavy)

✗ DON'T Fuse When:
• Intermediate results needed elsewhere
• Fused kernel becomes too complex
• Register pressure becomes too high
• Operations have different access patterns
```

In [ ]:
# Example: Sometimes intermediate results are needed

def compute_with_intermediate(x, y):
    """Case where intermediate result is needed."""
    # z1 = x + y (needed for both z2 AND for debugging/logging)
    # z2 = z1 * 2
    # z3 = z1 + 1
    # In this case, z1 is used twice, so we might keep it separate
    pass

# If z1 is only used for z2, then fuse: z2 = (x + y) * 2
@cuda.jit
def can_fuse(x, y, z2, n):
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(tid, n, stride):
        z2[i] = (x[i] + y[i]) * 2  # No intermediate storage

# If z1 is used multiple times, maybe keep separate
@cuda.jit
def keep_separate(x, y, z1, n):
    tid = cuda.grid(1)
    stride = cuda.gridsize(1)
    for i in range(tid, n, stride):
        z1[i] = x[i] + y[i]  # Store intermediate

# Then z1 can be used by multiple downstream kernels

---

## Part 6: Comprehensive Benchmark

In [ ]:
def run_fusion_benchmark(n, iterations=50):
    """Compare fused vs non-fused for various operation chains."""
    results = []
    
    x = np.random.rand(n).astype(np.float32)
    y = np.random.rand(n).astype(np.float32)
    z = np.random.rand(n).astype(np.float32)
    
    d_x = cuda.to_device(x)
    d_y = cuda.to_device(y)
    d_z = cuda.to_device(z)
    
    blocks, threads = 256, 256
    
    # Test 1: a*x + b*y + c
    d_temp1 = cuda.device_array(n, dtype=np.float32)
    d_temp2 = cuda.device_array(n, dtype=np.float32)
    d_out = cuda.device_array(n, dtype=np.float32)
    
    # Non-fused
    cuda.synchronize()
    start = time.perf_counter()
    for _ in range(iterations):
        scalar_multiply[blocks, threads](2.0, d_x, d_temp1, n)
        scalar_multiply[blocks, threads](3.0, d_y, d_temp2, n)
        vector_add[blocks, threads](d_temp1, d_temp2, d_out, n)
    cuda.synchronize()
    nonfused = (time.perf_counter() - start) / iterations * 1000
    
    # Fused
    @cuda.jit
    def fused_2axpy(a, x, b, y, out, n):
        tid = cuda.grid(1)
        stride = cuda.gridsize(1)
        for i in range(tid, n, stride):
            out[i] = a * x[i] + b * y[i]
    
    cuda.synchronize()
    start = time.perf_counter()
    for _ in range(iterations):
        fused_2axpy[blocks, threads](2.0, d_x, 3.0, d_y, d_out, n)
    cuda.synchronize()
    fused = (time.perf_counter() - start) / iterations * 1000
    
    results.append(('a*x + b*y', 3, nonfused, fused))
    
    # Test 2: sqrt(x*x + y*y)  (distance)
    @cuda.jit
    def nonfused_dist_step1(x, out, n):
        tid = cuda.grid(1)
        stride = cuda.gridsize(1)
        for i in range(tid, n, stride):
            out[i] = x[i] * x[i]
    
    @cuda.jit
    def nonfused_dist_sqrt(x, out, n):
        tid = cuda.grid(1)
        stride = cuda.gridsize(1)
        for i in range(tid, n, stride):
            out[i] = math.sqrt(x[i])
    
    cuda.synchronize()
    start = time.perf_counter()
    for _ in range(iterations):
        nonfused_dist_step1[blocks, threads](d_x, d_temp1, n)
        nonfused_dist_step1[blocks, threads](d_y, d_temp2, n)
        vector_add[blocks, threads](d_temp1, d_temp2, d_out, n)
        nonfused_dist_sqrt[blocks, threads](d_out, d_temp1, n)
    cuda.synchronize()
    nonfused = (time.perf_counter() - start) / iterations * 1000
    
    @cuda.jit
    def fused_distance(x, y, out, n):
        tid = cuda.grid(1)
        stride = cuda.gridsize(1)
        for i in range(tid, n, stride):
            out[i] = math.sqrt(x[i]*x[i] + y[i]*y[i])
    
    cuda.synchronize()
    start = time.perf_counter()
    for _ in range(iterations):
        fused_distance[blocks, threads](d_x, d_y, d_out, n)
    cuda.synchronize()
    fused = (time.perf_counter() - start) / iterations * 1000
    
    results.append(('sqrt(x²+y²)', 4, nonfused, fused))
    
    return results

# Run benchmark
n = 10_000_000
results = run_fusion_benchmark(n)

print(f"\nFusion Benchmark (N={n:,})")
print(f"{'='*60}")
print(f"{'Operation':<20} | {'Kernels':<8} | {'Non-fused':<10} | {'Fused':<10} | {'Speedup':<8}")
print(f"{'-'*60}")

for op, kernels, nf_time, f_time in results:
    speedup = nf_time / f_time
    print(f"{op:<20} | {kernels:<8} | {nf_time:<10.3f} | {f_time:<10.3f} | {speedup:<8.2f}x")

---

## Exercises

### Exercise 1: Fused Normalization

In [ ]:
# TODO: Implement fused min-max normalization
# normalized = (x - min) / (max - min)
# Given pre-computed min and max values

@cuda.jit
def fused_minmax_normalize(x, min_val, max_val, out, n):
    """Normalize x to [0, 1] using pre-computed min and max."""
    pass

# Test: x = [0, 5, 10], min=0, max=10
# Expected: [0, 0.5, 1.0]

### Exercise 2: Fused Gradient Descent Step

In [ ]:
# TODO: Implement gradient descent update with momentum
# velocity = momentum * velocity - lr * gradient
# weights = weights + velocity

@cuda.jit
def fused_sgd_momentum(weights, gradients, velocity, lr, momentum, n):
    """SGD with momentum update (modifies weights and velocity in-place)."""
    pass

# This fuses 4 operations into 1 kernel!

### Exercise 3: Fused Image Processing

In [ ]:
# TODO: Implement fused brightness + contrast adjustment
# output = clamp((input - 0.5) * contrast + 0.5 + brightness, 0, 1)

@cuda.jit
def fused_brightness_contrast(image, brightness, contrast, out, n):
    """Apply brightness and contrast adjustment in one kernel."""
    pass

# Test with various brightness/contrast values

---

## Summary

### Kernel Fusion Guidelines

```
1. IDENTIFY fusion opportunities:
   • Chains of element-wise operations
   • Same data accessed multiple times
   • Temporary arrays only used once

2. MEASURE before and after:
   • Time both approaches
   • Calculate memory traffic reduction
   • Check correctness!

3. BALANCE complexity:
   • Readable code matters
   • Too much fusion → register pressure
   • Document what's fused
```

### Memory Traffic Reduction

| Chain Length | Non-fused Reads/Writes | Fused Reads/Writes | Reduction |
|--------------|------------------------|--------------------|-----------|
| 2 ops | 4N | 2N | 2x |
| 3 ops | 6N | 2N | 3x |
| 4 ops | 8N | 2N | 4x |
| k ops | 2kN | 2N | kx |

### Key Takeaways

1. **Fusion dramatically reduces memory traffic**
2. **Fewer kernel launches = less overhead**
3. **GPUs are memory-bound for simple ops**
4. **Fusion is free performance**

---

## Week 3 Complete! 🎉

### What You Learned

| Day | Topic | Key Skills |
|-----|-------|------------|
| 1 | Grid-Stride Loops | Handle any size, professional patterns |
| 2 | Element-wise Ops | Math functions, benchmarking |
| 3 | SAXPY & BLAS | Memory bandwidth, standard ops |
| 4 | Fused Operations | Reduce traffic, combine kernels |

### Next Steps

📋 **Day 5:** Complete the checkpoint quiz

📋 **Week 4 Preview:** Reduction & Atomics
- Parallel reduction patterns
- Warp-level primitives
- Atomic operations
- Histogram computation